Here, I model the fund as a learning agent, which is able to adjust its demand function over time

Imagine the whole reinforcement mechanism only as the demand function of the LearningFund.

In [102]:
%matplotlib inline

import sys, os
import itertools
import collections

import tensorflow as tf
import numpy as np

import sklearn.pipeline
import sklearn.preprocessing
from sklearn.kernel_approximation import RBFSampler

import scipy
from scipy import optimize

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")
current_palette = sns.color_palette()

In [103]:
from thurner_model import NoiseTrader, Fund, DynamicFund, find_equilibrium 
import plotting

In [104]:
class Env:
    """
    Docstring
    """
    # Define our fundamental value V and asset-supply N
    V = 1.
    N = 1000.
    
    initial_price = 1.
    
    # Define noise trader (uses the NoiseTrader Class defined in thurner_model.py)
    roh_nt = 0.99
    sigma_nt = 0.035
    noise_trader = NoiseTrader(roh_nt, sigma_nt, V, N)
    initial_nt_spending = V*N
    
    def __init__(self):
        # tracks trader spending
        
        self.p_t = self.initial_price
        self.xi_t = self.initial_nt_spending
        self.done = False 
        
    # when resetting the environment, we set the state back to the initial one 
    def reset(self):
        self.p_t = self.initial_price
        self.xi_t = self.initial_nt_spending

    def step(self, funds):
        """Finds equilibrium, and updates environment parameters""" 
        # track the old price for the investor mechanism
        p_tm1 = self.p_t
        
        # 1. Find the new price for the timestep
        self.xi_t = self.noise_trader.cash_spending(self.xi_t)
        self.p_t = find_equilibrium(self.xi_t, funds)
    
        # 2. update the holdings of all the funds (wealth, shares and cash)
        current_wealth = []
        
        for fund in funds:
            fund.update_holdings(self.p_t)
            fund.check_and_make_bankrupt(self.p_t)
            
            fund.process_inflows(p_tm1, self.p_t)
            
            new_wealth_of_fund = fund.get_wealth(self.p_t)
            current_wealth.append(new_wealth_of_fund)
            
            # set done to True if one fund increases its wealth 50-fold
            if new_wealth_of_fund > 50*fund.initial_wealth:
                self.done = True
                
        return current_wealth

In [105]:
env = Env()

In [106]:
# Create examples for observations to train the featurizer
tracked_fund = DynamicFund(25)
other_funds = [DynamicFund((i+1)*5) for i in range(9)]
other_funds.append(tracked_fund)
print([f.beta for f in other_funds])

states = np.zeros((10000,5))
for i in range(10000):
    current_wealth = env.step(other_funds)
    states[i] = np.array([env.p_t,
                         tracked_fund.get_wealth(env.p_t),
                         tracked_fund.cash,
                         tracked_fund.shares,
                         tracked_fund.activation_delay])
    # record the state of the fund

[5, 10, 15, 20, 25, 30, 35, 40, 45, 25]


In [107]:
# (By Denny Britz)
# Feature Preprocessing: Normalize to zero mean and unit variance
# We use a few samples from the observation space to do this

observation_examples = states 
scaler = sklearn.preprocessing.StandardScaler()
scaler.fit(observation_examples)

# Used to converte a state to a featurized represenation.
# We use RBF kernels with different variances to cover different parts of the space
featurizer = sklearn.pipeline.FeatureUnion([
        ("rbf1", RBFSampler(gamma=5.0, n_components=100)),
        ("rbf2", RBFSampler(gamma=2.0, n_components=100)),
        ("rbf3", RBFSampler(gamma=1.0, n_components=100)),
        ("rbf4", RBFSampler(gamma=0.5, n_components=100))
        ])
featurizer.fit(scaler.transform(observation_examples))

FeatureUnion(n_jobs=1,
       transformer_list=[('rbf1', RBFSampler(gamma=5.0, n_components=100, random_state=None)), ('rbf2', RBFSampler(gamma=2.0, n_components=100, random_state=None)), ('rbf3', RBFSampler(gamma=1.0, n_components=100, random_state=None)), ('rbf4', RBFSampler(gamma=0.5, n_components=100, random_state=None))],
       transformer_weights=None)

In [108]:
def featurize_state(state):
    """
    Returns the featurized representation for a state.
    """
    scaled = scaler.transform([state])
    featurized = featurizer.transform(scaled)
    return featurized[0]

In [109]:
class LearningFund(DynamicFund):
    """
    Description: Base Class is DynamicFund, defined in thurner_model.py
    The Learning Fund learns its demand function via reinforcement learning
    """
    
    def __init__(self):
        # The learning fund does not need a beta
        
        self.cash = self.initial_wealth
        self.shares = 0 
        self.activation_delay = 0
        
        self.performance = 0.0
        self.previous_wealth = self.initial_wealth
        self.previous_investment = 0.0
   
    def get_state(self, p_t):
        # The state is composed of the current price, as well as the current
        # holdings of the fund
        state = np.array([p_t,
                          self.get_wealth(p_t), 
                          self.cash,
                          self.shares,
                          self.activation_delay])
        return state

    def get_demand(self, p_t): # this needs to be a function of p_t for the market clearing to work
        """
        Args:
            p_t : current_price in the environment
        Returns:
            A number for the learning_fund's demand, estimated by the
            policy_estimator, based on the current state
        """ 
        state = self.get_state(p_t)
        
        return policy_estimator.predict(state)

In [110]:
class PolicyEstimator():
    """
    Policy Function approximator. Also called Actor.
    """
    
    def __init__(self, learning_rate=0.01, scope="policy_estimator"):
        with tf.variable_scope(scope):
            self.state = tf.placeholder(tf.float32, [400], "state")
            self.target = tf.placeholder(dtype=tf.float32, name="target")

            # This is just linear classifier
            self.mu = tf.contrib.layers.fully_connected(
                inputs=tf.expand_dims(self.state, 0),
                num_outputs=1,
                activation_fn=None,
                weights_initializer=tf.zeros_initializer)
            self.mu = tf.squeeze(self.mu)
            
            self.sigma = tf.contrib.layers.fully_connected(
                inputs=tf.expand_dims(self.state, 0),
                num_outputs=1,
                activation_fn=None,
                weights_initializer=tf.zeros_initializer)
            
            self.sigma = tf.squeeze(self.sigma)
            self.sigma = tf.nn.softplus(self.sigma) + 1e-5
            self.normal_dist = tf.contrib.distributions.Normal(self.mu,
                                                               self.sigma)
            self.demand = self.normal_dist._sample_n(1)
            
            # clip the demand, maximum demand is given by:
            max_demand = learning_fund.lambda_max * \
                            learning_fund.get_wealth(env.p_t) / env.p_t
                
            self.demand = tf.clip_by_value(self.demand,
                                           0,
                                           max_demand)

            # Loss and train op
            self.loss = -self.normal_dist.log_prob(self.demand) * self.target
            # Add cross entropy cost to encourage exploration
            self.loss -= 1e-1 * self.normal_dist.entropy()
            
            self.optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
            self.train_op = self.optimizer.minimize(
                self.loss, global_step=tf.contrib.framework.get_global_step())
    
    def predict(self, state, sess=None):
        sess = sess or tf.get_default_session()
        state = featurize_state(state)
        return sess.run(self.demand, { self.state: state })

    def update(self, state, target, demand, sess=None):
        sess = sess or tf.get_default_session()
        state = featurize_state(state)
        feed_dict = { self.state: state,
                      self.target: target,
                      self.demand: demand  }
        _, loss = sess.run([self.train_op, self.loss], feed_dict)
        return loss

In [111]:
class ValueEstimator():
    """
    Value Function approximator. Also called Critic.
    """
    
    def __init__(self, learning_rate=0.1, scope="value_estimator"):
        with tf.variable_scope(scope):
            self.state = tf.placeholder(tf.float32, [400], "state")
            self.target = tf.placeholder(dtype=tf.float32, name="target")

            # This is just a linear classifier
            self.output_layer = tf.contrib.layers.fully_connected(
                inputs=tf.expand_dims(self.state, 0),
                num_outputs=1,
                activation_fn=None,
                weights_initializer=tf.zeros_initializer)

            self.value_estimate = tf.squeeze(self.output_layer)
            self.loss = tf.squared_difference(self.value_estimate, self.target)

            self.optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
            self.train_op = self.optimizer.minimize(
                self.loss, global_step=tf.contrib.framework.get_global_step())        
    
    def predict(self, state, sess=None):
        sess = sess or tf.get_default_session()
        state = featurize_state(state)
        return sess.run(self.value_estimate, { self.state: state })

    def update(self, state, target, sess=None):
        sess = sess or tf.get_default_session()
        state = featurize_state(state)
        feed_dict = { self.state: state, self.target: target }
        _, loss = sess.run([self.train_op, self.loss], feed_dict)
        return loss

In [117]:
def actor_critic(env, policy_estimator, value_estimator, num_episodes,
                 discount_factor=1.0):
    """
    Actor Critic Algorithm. Optimizes the policy 
    function approximator using policy gradient.
    
    Args:
        env: My self created environment, specified above.
        policy_estimator: Policy Function to be optimized 
        value_estimator: Value function approximator, used as a critic
        num_episodes: Number of episodes to run for
        discount_factor: Time-discount factor
    
    Returns:
        An EpisodeStats object with two numpy arrays for episode_lengths and
        episode_rewards.
    """
    
    # Keeps track of useful statistics
    stats = plotting.EpisodeStats(
        episode_lengths=np.zeros(num_episodes),
        episode_rewards=np.zeros(num_episodes))    
    
    Transition = collections.namedtuple("Transition", ["state", "action",
                                                       "reward", "next_state",
                                                       "done"])
    
    for i_episode in range(num_episodes):
        
        # Reset everything
        prices = []
        funds_wealth = []

        # Create the funds 
        number_of_funds = 10
        funds = [Fund((i+1)*5) for i in range(number_of_funds)]
        
        # Add our learning fund
        funds.append(learning_fund)

        # Reset the environment 
        env.reset() 
        
        episode = []
        
        # One step in the environment
        for t in itertools.count():
            
            # get the demand (which is our action) of the learning fund
            demand = learning_fund.get_demand(env.p_t) 
            
            state = learning_fund.get_state(env.p_t)
            
            # Simulate a step in the environment,
            # record the wealth of all funds in current_wealth
            current_wealth = env.step(funds)
            
            # record the wealth of all funds and the current price
            funds_wealth.append(current_wealth)
            prices.append(env.p_t)
            
            # we assume one learning fund for the moment
            next_state = np.array([env.p_t,
                                   learning_fund.get_wealth(env.p_t), 
                                   learning_fund.cash,
                                   learning_fund.shares,
                                   learning_fund.activation_delay]) 

            reward = learning_fund.performance 
            
            # Keep track of the transition
            episode.append(Transition(state=state, action=demand,
                                      reward=reward, next_state=next_state,
                                      done=env.done))
            
            # Update statistics
            stats.episode_rewards[i_episode] += reward
            stats.episode_lengths[i_episode] = t
            
            # Calculate TD Target
            value_next = value_estimator.predict(next_state)
            td_target = reward + discount_factor * value_next
            td_error = td_target - value_estimator.predict(state)
            
            # Update the value estimator
            value_estimator.update(state, td_target)
            
            # Update the policy estimator
            # using the td error as our advantage estimate
            policy_estimator.update(state, td_error, demand)
            
            # Print out which step we're on, useful for debugging.
            print("\rStep {} @ Episode {}/{} ({})".format(
                    t, i_episode + 1, num_episodes,
                    stats.episode_rewards[i_episode - 1]), end="")
            
            # env.done is True if one fund increases its wealth 50-fold
            if env.done:
                break
                
            state = next_state
    
    return stats

In [118]:
# Create the environment
env = Env()

# Create our learning_fund
learning_fund = LearningFund()


##############################################################################
# This works
tf.reset_default_graph()

global_step = tf.train.get_global_step()
policy_estimator = PolicyEstimator(learning_rate=0.001)
value_estimator = ValueEstimator(learning_rate=0.1)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    # Due to randomness in the policy, the number of episodes you need varies
    # TODO: Sometimes the algorithm gets stuck, I'm not sure what exactly is
    # happening there.
    stats = actor_critic(env, policy_estimator, value_estimator,
                         num_episodes=50, discount_factor=0.95)

Step 1196 @ Episode 1/50 (0.0)

KeyboardInterrupt: 